In [ ]:
import wikipedia
import requests
import bs4
import pymongo
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import bs4 as BeautifulSoup
import urllib.request  
from mttkinter import *  
from tkinter import *

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client.textbook
col = db.index

def _create_dictionary_table(text_string) -> dict:
   
    #removing stop words
    stop_words = set(stopwords.words("english"))
    
    words = word_tokenize(text_string)
    
    #reducing words to their root form
    stem = PorterStemmer()
    
    #creating dictionary for the word frequency table
    frequency_table = dict()
    for wd in words:
        wd = stem.stem(wd)
        if wd in stop_words:
            continue
        if wd in frequency_table:
            frequency_table[wd] += 1
        else:
            frequency_table[wd] = 1

    return frequency_table

def _calculate_sentence_scores(sentences, frequency_table) -> dict:   

    #algorithm for scoring a sentence by its words
    sentence_weight = dict()

    for sentence in sentences:
        sentence_wordcount = (len(word_tokenize(sentence)))
        sentence_wordcount_without_stop_words = 0
        for word_weight in frequency_table:
            if word_weight in sentence.lower():
                sentence_wordcount_without_stop_words += 1
                if sentence[:7] in sentence_weight:
                    sentence_weight[sentence[:7]] += frequency_table[word_weight]
                else:
                    sentence_weight[sentence[:7]] = frequency_table[word_weight]

        sentence_weight[sentence[:7]] = sentence_weight[sentence[:7]] / sentence_wordcount_without_stop_words

       

    return sentence_weight

def _calculate_average_score(sentence_weight) -> int:
   
    #calculating the average score for the sentences
    sum_values = 0
    for entry in sentence_weight:
        sum_values += sentence_weight[entry]

    #getting sentence average value from source text
    average_score = (sum_values / len(sentence_weight))

    return average_score

def _get_article_summary(sentences, sentence_weight, threshold):
    sentence_counter = 0
    article_summary = ''

    for sentence in sentences:
        if sentence[:7] in sentence_weight and sentence_weight[sentence[:7]] >= (threshold):
            article_summary += " " + sentence
            sentence_counter += 1

    return article_summary

def _run_article_summary(article):
    
    #creating a dictionary for the word frequency table
    frequency_table = _create_dictionary_table(article)

    #tokenizing the sentences
    sentences = sent_tokenize(article)

    #algorithm for scoring a sentence by its words
    sentence_scores = _calculate_sentence_scores(sentences, frequency_table)

    #getting the threshold
    threshold = _calculate_average_score(sentence_scores)

    #producing the summary
    article_summary = _get_article_summary(sentences, sentence_scores, 1.5 * threshold)

    return article_summary


#index = input("Enter search term : ")
#l = []
#l.append(index)

#fh1 = open('topics.txt','r')
#l = fh1.readlines()
#fh1.close()

def summarizeFunc():
    for z in range(0,len(topicsToSearch)):

        index =(topicsToSearch[z].strip()).lower()
        print(index)
        if(index!=''):
            try :

                dis = col.find_one({"topic": index},{"_id": 0})
                fh = open('demo2.txt', "a+", encoding = "utf-8")
            
                count1 = 0
                for vals in dis.values():

                    if(count1==0):
                        count1=count1+1
                        vals=vals.upper()
                        #print(vals)
                        fh.write(vals)
                    else:
                        article_content = ''
                        article_content+=vals
                        #print(article_content)
                        if __name__ == '__main__':
                            summary_results = _run_article_summary(article_content)
                            #print(summary_results)

                        fh.write('\n')
                        fh.write('\n')
                        fh.write(summary_results)
                        fh.write('\n')
                        fh.write('\n')
                        fh.write('\n')

                fh.close()
                print("Already in database")


            except:

                print('Not in DB')
                try:

                    src = wikipedia.page(index)
                    print(src)
                    print(src.url)

                except:
                    print(index+" no url found ")
                    continue 

                res = requests.get(src.url)


                soup = bs4.BeautifulSoup(res.text,'html.parser')
                #print(soup)

                fh = open('demo1.txt', "w+", encoding = "utf-8") 
                #for j in soup.select('h'):
                #    print(j.text)
                #count = 0
                fh.write(index)
                fh.write('\n')
                for i in soup.select('p'):
                    #if(count == 5):
                     #   break;
                    #else:
                    #type(i.text)
                    fh.write(i.text)
                    #print(i.text)
                    #count = count + 1
                fh.close()


                fh = open('demo1.txt', "r", encoding = "utf-8")

                t1 = fh.read()
                t1 = re.sub("[\[].*?[\]]", "", t1)
                t1 = re.sub(' {[^}]*}','',t1)
                #print(type(t1))
                dict1 = {"topic": index, "value": t1}
                col.insert_one(dict1)
                fh.close()

                #query = {"name": index}
                #doc = col.find(query)
                #for x in doc:
                #    print(x)

                fh = open('demo2.txt', "a+", encoding = "utf-8")
                #pprint.pprint(col.find_one({"name": index},{"value": 1}))
                dis = col.find_one({"topic": index},{"_id": 0})
                count1=0
                for vals in dis.values():
                #for i in range(0,1):

                    if(count1==0):
                        count1=count1+1
                        vals=vals.upper()
                        #print(vals)
                        fh.write(vals)
                    else:
                        article_content = ''
                        article_content+=vals
                        #print(article_content)

                        if __name__ == '__main__':
                            summary_results = _run_article_summary(article_content)
                            #print(summary_results)

                        fh.write('\n')
                        fh.write('\n')
                        fh.write(summary_results)
                        fh.write('\n')
                        fh.write('\n')
                        fh.write('\n')
                fh.close()
        else: 
            continue 


def expand():
    
    for i in range(int(subtopicNum.get())):
        temp=StringVar()
        subtopicLabel = Label(parent,text = "Subtopic "+str(i+1)+':',background='#00074f',foreground='white',font = "Helvetica 20 bold").grid(row = 5+i, column = 0,pady=20,padx=20) 
        subtopicEntry = Entry(parent,textvariable=temp,font = "Helvetica 20 bold").grid(row = 5+i, column = 1,pady=20,padx=20) 
        subtopicList.append(temp)
    generate = Button(parent,bg='white', text = "Generate Book", command=printBook,font = "Helvetica 20 bold")
    generate.grid(row = int(subtopicNum.get())+5 , column = 0,pady=20,padx=20)  

def printBook():
    topicsToSearch.append(titleName.get())
    for i in subtopicList:
        topicsToSearch.append(i.get())
    print(topicsToSearch)
    summarizeFunc()

parent = mtTkinter.Tk()  
parent.configure(background='#00074f')

titleName=StringVar()
subtopicNum=StringVar()
subtopicList=[]
topicsToSearch=[] #this is the list that is printing so use this to search and create document 
#just put a for loop over your code and feed what ever is in the list 

titleLabel = Label(parent,background='#00074f',foreground='white',text = "Title",font = "Helvetica 20 bold").grid(row = 0, column = 0,pady=20,padx=20) 
subtopicLabel = Label(parent,background='#00074f',foreground='white',text = "Number of subtopics",font = "Helvetica 20 bold").grid(row = 1, column = 0,pady=20,padx=20) 

titleEntry = Entry(parent,textvariable=titleName,font = "Helvetica 20 bold").grid(row = 0, column = 1,pady=20,padx=20)  
subtopicEntry = Entry(parent,textvariable=subtopicNum,font = "Helvetica 20 bold").grid(row = 1, column = 1,pady=20,padx=20)  

expand = Button(parent, text = "Expand",bg='white', command=expand,font = "Helvetica 20 bold")
expand.grid(row = 4, column = 0,pady=20,padx=20)  

parent.mainloop()  


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\91996\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-1-1c5726ce2764>", line 236, in expand
    for i in range(int(subtopicNum.get())):
ValueError: invalid literal for int() with base 10: ''
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\91996\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-1-1c5726ce2764>", line 236, in expand
    for i in range(int(subtopicNum.get())):
ValueError: invalid literal for int() with base 10: ''


['sachin tendulkar', 'world cup', 'cups']
sachin tendulkar
Already in database
world cup
Not in DB
<WikipediaPage 'FIFA World Cup'>
https://en.wikipedia.org/wiki/FIFA_World_Cup
cups
Not in DB
<WikipediaPage 'CUPS'>
https://en.wikipedia.org/wiki/CUPS


In [8]:
dis = col.find_one({"topic": 'linear regression'},{"_id": 0})
for vals in dis.values():
    print(vals)

linear regression
linear regression
In statistics, linear regression is a linear approach to modeling the relationship between a scalar response (or dependent variable) and one or more explanatory variables (or independent variables). The case of one explanatory variable is called simple linear regression. For more than one explanatory variable, the process is called multiple linear regression. This term is distinct from multivariate linear regression, where multiple correlated dependent variables are predicted, rather than a single scalar variable.
In linear regression, the relationships are modeled using linear predictor functions whose unknown model parameters are estimated from the data. Such models are called linear models. Most commonly, the conditional mean of the response given the values of the explanatory variables (or predictors) is assumed to be an affine function of those values; less commonly, the conditional median or some other quantile is used. Like all forms of regres